## Imports and settings

Change the `patches_path` to a local location if you like, this is a good idea when you are training for a long time and need to access the same file a few times, otherwise, e.g. when only evaluating, the copy-overhead to do so is not worth it.

In [25]:
from google.colab import drive

drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
import re
import os
import cv2
import time
import numpy
import pickle
import numbers
import warnings
from glob import glob
from skimage.io import imread, imsave
from scipy.ndimage.measurements import label

import matplotlib
from matplotlib import pyplot

from sklearn.model_selection import train_test_split

from keras.optimizers import Adam
from keras.models import load_model, Model
from keras.layers import *

import keras.backend as K
import tensorflow as tf
from keras.backend.common import epsilon

try:
  from jupyter_progressbar import ProgressBar
except ImportError:
  !pip3 install jupyter_progressbar
  from jupyter_progressbar import ProgressBar

In [0]:
# root_path = '/content/gdrive/My Drive/Eye_in_the_sky/' # Leslie

root_path = '/content/gdrive/My Drive/Projects/2019/Eye_in_the_sky/' # Herbert

image_path = f'{root_path}Training Data/Images'
mask_path = f'{root_path}Training Data/Mask Labels'
gdrive_patches_path = f'{root_path}Training Data/Patches/v2'
results_path = f'{root_path}Results'

# models_path = f'{root_path}Training Data/Models'

patches_path = gdrive_patches_path

# Only if you use the next cell below to copy the patches to this directory.
# The copying takes some time, but the data loads approx. 3x faster. This is
# particularly good when training.

# patches_path = '/content/patches'


# How much of the patch should be defined, since parts of the satelite images
# are black
define_t = 0.9

In [0]:
# !mkdir '{patches_path}'
# for sample in ProgressBar(os.listdir(f'{gdrive_patches_path}')):
#   !cp '{gdrive_patches_path}/{sample}' '{patches_path}'

## Data

Creates train, validation and test split

Defines `iter_super_batches` to load multiple filenames at once for trianing, and loads the validation set.

Test data is not loaded to save memory during training, but kept seperately to ultimately be able to create a proper test report.

Also normalizes the data for the neural network to values between -2 and 2.



### Data splitting

Train / test split, done on the basis of the original images, not the patches as they may overlap.

Note that I fixed the test and validation set, such that loading (partially) trained models later on can not accidentally overlap train en test+validaiton sets.

In [0]:
# samples will contain the extension-less filenames for the images that have
# shapes and shapes that have images.

image_filenames = os.listdir(f'{image_path}')
mask_filenames = os.listdir(f'{mask_path}')

samples = numpy.array(list({
    filename[:-4]
    for filename in image_filenames
    if filename.endswith('.tif')
} & {
    filename[:-4]
    for filename in mask_filenames
    if filename.endswith('.tif')
}))

samples = sorted(samples)

train_samples, test_samples = train_test_split(samples, test_size=0.05, random_state = 597213461)

In [0]:
train_batches = sum(
    (glob(f'{patches_path}/sample-{sample}_batch-*.p3')
     for sample in train_samples), 
    [])

test_batches_ = sum(
    (glob(f'{patches_path}/sample-{sample}_batch-*.p3')
     for sample in test_samples), 
    [])

In [0]:
validation_batches = [
 f'{patches_path}/sample-S2B1C_20180927_108_32UMC_TOA_10_batch-0.p3',
 f'{patches_path}/sample-S2B1C_20180927_108_32UMC_TOA_10_batch-1.p3',
 f'{patches_path}/sample-S2B1C_20180927_108_32UMC_TOA_10_batch-2.p3',
 f'{patches_path}/sample-S2B1C_20180927_108_32UMC_TOA_10_batch-3.p3',
 f'{patches_path}/sample-S2B1C_20180927_108_32UMC_TOA_10_batch-4.p3',
 f'{patches_path}/sample-S2B1C_20180927_108_32UMC_TOA_10_batch-5.p3',
 f'{patches_path}/sample-S2B1C_20180927_108_32UMC_TOA_10_batch-6.p3',
 f'{patches_path}/sample-S2B1C_20180927_108_32UMC_TOA_10_batch-7.p3',
 f'{patches_path}/sample-S2B1C_20180927_108_32UMC_TOA_10_batch-8.p3',
]
test_batches = [
 f'{patches_path}/sample-S2A1C_20180912_108_32UNA_TOA_10_batch-0.p3',
 f'{patches_path}/sample-S2A1C_20180912_108_32UNA_TOA_10_batch-1.p3',
 f'{patches_path}/sample-S2A1C_20180912_108_32UNA_TOA_10_batch-2.p3',
 f'{patches_path}/sample-S2A1C_20180912_108_32UNA_TOA_10_batch-3.p3',
 f'{patches_path}/sample-S2B1C_20180702_008_31UFV_TOA_10_batch-0.p3',
 f'{patches_path}/sample-S2A1C_20180918_051_32ULD_TOA_10_batch-0.p3',
 f'{patches_path}/sample-S2A1C_20180918_051_32ULD_TOA_10_batch-1.p3',
 f'{patches_path}/sample-S2A1C_20180918_051_32ULD_TOA_10_batch-2.p3',
 f'{patches_path}/sample-S2A1C_20180918_051_32ULD_TOA_10_batch-3.p3'
]

assert set(test_batches + validation_batches) == set(test_batches_), "Something changed in the test train split, such that these constants are not trustworthy anymore. Rethink/do your train/validation/test data splitting."
assert len(set(test_batches + validation_batches) & set(train_batches)) == 0, "Train data overlaps with test or validation data, such that these constants are REALLY not trustworthy anymore. Rethink/do your train/validation/test data splitting."

### Helper functions

In [0]:
# One could also inspect all files, but they are large and opening them takes a
# long time. Hence the values were precomputed in the same notebook as where
# the patches are created.

with open(f'{gdrive_patches_path}/../v2-lengths.p3', 'rb') as f:
    lengths = pickle.load(f)

def get_n_samples(filename, t):
  """The `v2-lengths.p3` file contains the number of samples in each patches
  file given a threshold of how many pixels are defined. That is, some of the
  satelite images are partially black (or undefined). If filename is not a
  `str`, returns how many samples in that patches file have at least `t`
  defined. If filename is a list (of `str`), returns the sum of samples for
  each filename in the list."""
  
  assert type(filename) in {str, list}
  if type(filename) == list:
    return sum(get_n_samples(filename, t) for filename in filename)
  counts = lengths[filename]
  keys = [k for k in counts.keys() if k >= t]
  if len(keys) == 0:
    return 0
  return counts[min(keys)]

In [0]:
# The offsets were defined as the medians in `Estimate mean and stddev`. There were some extreme
# outliers, so 5% of the large outliers was neglected at both sides (so 10% in total). Moreover,
# Only 1% of the data was used finding the outlier thresholds, since the resolution of the images
# are very high and sorting gets very slow.
#
# Furthermore, values are clipped between -2 and 2, since neural network training is not stable
# w.r.t. extreme values.
#
# This code can take up to a few seconds for larger batches, within the order of maginitude of
# loading a batch, or even slower. It might be better to do the preprocessing while creating the
# batches, which was originally the plan, but I forgot to do so. This way we could also store
# the patch as float32 instead of uint64, halving the space needed and hence halving pickle
# loading time.

# TODO: move to patch creation


def preprocess_input(image):
  """ scales the pixels to have mean 0 and stdev 1 and furthermore, clips them between values -2 and
  2, for neural network training"""
  offsets = numpy.array([
      1264.06439133,  993.71959535,  879.00178412,  740.15150524,  990.24213408,
      1798.24382807, 2154.90345801, 2121.7030217 ,  695.56159991,   11.36244131,
      1707.93913123, 1008.18696351])
  factors = numpy.array([
       92.55388916, 139.58717587, 193.49003888, 319.68365431, 317.27768991,
      444.25386892, 565.17997105, 593.66462148, 160.98605446,   2.23359842,
      631.6088602 , 520.73160082])

  image -= offsets[(numpy.newaxis, ) * (len(image.shape) - 1)]
  for i, f in enumerate(factors):
      image[..., i] /= f

  numpy.clip(image, -2, 2, image)
  return image

In [0]:
def iter_filenames(filenames, shuffle=True):
  """Infinitively iterates through the filenames, in an epoch manor, that is all filenames need to be
  yielded at least n-1 time to yield another filename for the n-th time. They can either be shuffled,
  or each epoch can have the order as `filename`m depending on `shuffle`"""
  
  filenames = numpy.array(filenames)
  order = numpy.arange(len(filenames))
  while True:
    if shuffle:
      numpy.random.shuffle(order)
    for filename in filenames[order]:
      yield filename


def take(it, n):
  """ Returns the next `n` items from `it`, or less if not available, as a list"""
  # zip ensures the list finishes whe`it` is empty or when `n` are extracted. Moreover,
  # range is the first in zip to prevent that `n+1` are extracted while only `n` are returned.
  return [x for _, x in zip(range(n), next(it))]


def iter_super_batches(filenames, files_at_once=25, shuffle=True, t=0.8, channels=slice(None)):
  """ Yields batches of the samples in `files_at_once` random `filenames`, and only selects
  samples with at least `t` pixels defined. The `channels` parameter doesn't work (yet) and should
  not be changed.
  @param `shuffle` only shuffles the filenames, not the samples themselves
                   (Keras'sfit can shuffle the samples).
  @yields a tuple of the images and the labels"""
  filenames = iter_filenames(filenames, shuffle=shuffle)
  
  while True:
    batch_filenames = take(filenames, files_at_once)
    n = get_n_samples([fn.split('/')[-1] for fn in batch_filenames], t)

    XX = numpy.zeros((n, 512, 512, 12), dtype=numpy.float32)
    yy = numpy.zeros((n, 512, 512, 1), dtype=numpy.bool)

    i = 0
    for filename in batch_filenames:
      with open(filename, 'rb') as f:
        data = pickle.load(f)

      d = data['defined'].mean(axis=1).mean(axis=1) > t
      X = preprocess_input(data['image'][d, ..., channels].astype(numpy.float32))
      y = data['buildings'][..., None][d]

      XX[i:i+len(y)] = X
      yy[i:i+len(y)] = y

      i+=len(y)

    yield XX, yy

### Validation data

Load all validation data, test data could be loaded in the same manor, but this fills up memory and would force you to train in smaller super batches later on.

In [0]:
n_validation_samples = sum(
    get_n_samples(fn.split('/')[-1], define_t)
    for fn in validation_batches)

In [36]:
# Load validation data

X_valid = numpy.zeros((n_validation_samples, 512, 512, 12), dtype=numpy.float32)
y_valid = numpy.zeros((n_validation_samples, 512, 512, 1), dtype=numpy.bool)


t = 0.9
i = 0
for filename in ProgressBar(validation_batches):
  with open(filename, 'rb') as f:
    data = pickle.load(f)
  d = data['defined'].mean(axis=1).mean(axis=1) > define_t
  X = preprocess_input(data['image'].astype(numpy.float32))[d]
  y = data['buildings'][..., None][d]

  X_valid[i:i+len(y)] = X
  y_valid[i:i+len(y)] = y

  i+=len(y)

  del X, y, d, data

### Validation Examples

In [37]:
for x, y, axis in zip(X_valid, y_valid, pyplot.subplots(8, 2, figsize=(25, 100))[1].ravel() ):
  rgb = (x[..., [3,2,1]] + 2) / 4
  mask = y[..., 1:]
  axis.imshow(rgb)
  axis.imshow(numpy.dstack([y, numpy.zeros(y.shape), numpy.zeros(y.shape), 0.8 * y]))
  
pyplot.tight_layout()

pyplot.savefig(f'{results_path}/example_training_mask.png')

Output hidden; open in https://colab.research.google.com to view.

## U-net model

### Model

In [0]:
class UNET(Model):
    def __init__(self, n_channels=12, *args, **kwargs):
        """Copied from https://github.com/zhixuhao/unet/blob/master/model.py"""
        inputs = Input([None, None, n_channels])

        nfilters = 16 # in the original architecture, this was 64

        conv1 = Conv2D(nfilters, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(inputs)
        conv1 = Conv2D(nfilters, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv1)
        pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)
        nfilters *= 2
        last_out = pool1

        conv2 = Conv2D(nfilters, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(last_out)
        conv2 = Conv2D(nfilters, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv2)
        pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)
        nfilters *= 2
        last_out = pool2

        conv3 = Conv2D(nfilters, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(last_out)
        conv3 = Conv2D(nfilters, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv3)
        pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)
        nfilters *= 2
        last_out = pool3

        conv4 = Conv2D(nfilters, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(last_out)
        conv4 = Conv2D(nfilters, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv4)
        drop4 = Dropout(rate=0.5)(conv4)
        pool4 = MaxPooling2D(pool_size=(2, 2))(drop4)
        nfilters *= 2
        last_out = pool4

        conv5 = Conv2D(nfilters, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(last_out)
        conv5 = Conv2D(nfilters, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv5)
        drop5 = Dropout(rate=0.5)(conv5)
        nfilters //= 2 # // is for integer division, where / is floating point division in Python 3
        last_out = drop5

        up6 = Conv2D(nfilters, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(last_out))
        merge6 = concatenate([drop4,up6], axis = 3)

        conv6 = Conv2D(nfilters, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge6)
        conv6 = Conv2D(nfilters, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv6)
        nfilters //= 2
        last_out = conv6

        up7 = Conv2D(nfilters, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(last_out))
        merge7 = concatenate([conv3,up7], axis = 3)
        conv7 = Conv2D(nfilters, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge7)
        conv7 = Conv2D(nfilters, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv7)
        nfilters //= 2
        last_out = conv7
        
        up8 = Conv2D(nfilters, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(last_out))
        merge8 = concatenate([conv2,up8], axis = 3)
        conv8 = Conv2D(nfilters, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge8)
        conv8 = Conv2D(nfilters, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv8)
        nfilters //= 2
        last_out = conv8

        up9 = Conv2D(nfilters, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(last_out))
        merge9 = concatenate([conv1,up9], axis = 3)

        conv9 = Conv2D(nfilters, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge9)
        conv9 = Conv2D(nfilters, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv9)
        conv9 = Conv2D(2, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv9)
        conv10 = Conv2D(1, 1, activation = 'sigmoid')(conv9)

        super(UNET, self).__init__(inputs=inputs, outputs=conv10)

### Metrics

In [0]:
def precision(target, output, epsilon = 0.000001):
  P = output > 0.5
  TP = tf.logical_and(P, target > 0)
  return (epsilon + K.sum(tf.cast(TP, tf.float32))) / (
      epsilon + K.sum(tf.cast(P, tf.float32)))


def recall(target, output, epsilon = 0.000001):
  P = target > 0
  TP = tf.logical_and(P, output > 0.5)
  return (epsilon + K.sum(tf.cast(TP, tf.float32))) / (
      epsilon + K.sum(tf.cast(P, tf.float32)))


def k_min(y_true, y_pred):
  return K.min(y_pred)


def k_max(y_true, y_pred):
  return K.max(y_pred)


def k_mean(y_true, y_pred):
  return K.mean(y_pred > 0.5)

### Loss functions

In [0]:
def _to_tensor(x, dtype):
    return tf.convert_to_tensor(x, dtype=dtype)
  

def normalized_binary_crossentropy(target, output, from_logits=False):
    if not from_logits:
        _epsilon = _to_tensor(epsilon(), output.dtype.base_dtype)
        output = tf.clip_by_value(output, _epsilon, 1 - _epsilon)
        output = tf.log(output / (1 - output))
    
    w0 = tf.cast(K.sum(target), tf.float32)
    s = K.shape(target)
    w1 = tf.cast(s[0] * s[1] * s[2] * s[3], tf.float32) - w0
    r = K.sqrt(w0*w0 + w1*w1)
    w0, w1 = w0 / r, w1 / r
    target2 = tf.cast(target, tf.float32)
    w = w0 * (1. - target2) + w1 * target2
    
    r = tf.nn.sigmoid_cross_entropy_with_logits(
        labels=target, logits=output)
    return w * r


def auto_weighting_binary_crossentropy(target, output, from_logits=False):
    if not from_logits:
        _epsilon = _to_tensor(epsilon(), output.dtype.base_dtype)
        output = tf.clip_by_value(output, _epsilon, 1 - _epsilon)
        output = tf.log(output / (1 - output))
    
    subsample = tf.cast(tf.logical_or(
      tf.less(K.random_uniform(K.shape(target), minval=0, maxval=1), 0.02),
      tf.greater(target, 0.5)
    ), tf.float32)
    
    r = tf.nn.sigmoid_cross_entropy_with_logits(
        labels=target, logits=output)
    return r * subsample / 0.04
  

def soft_dice_loss(y_true, y_pred, epsilon=1e-6): 
    ''' 
    Soft dice loss calculation for arbitrary batch size, number of classes, and number of spatial dimensions.
    Assumes the `channels_last` format.
  
    # Arguments
        y_true: b x X x Y( x Z...) x c One hot encoding of ground truth
        y_pred: b x X x Y( x Z...) x c Network output, must sum to 1 over c channel (such as after softmax) 
        epsilon: Used for numerical stability to avoid divide by zero errors
    
    # References
        V-Net: Fully Convolutional Neural Networks for Volumetric Medical Image Segmentation 
        https://arxiv.org/abs/1606.04797
        More details on Dice loss formulation 
        https://mediatum.ub.tum.de/doc/1395260/1395260.pdf (page 72)
        
        Adapted from https://github.com/Lasagne/Recipes/issues/99#issuecomment-347775022
    '''
    
    # skip the batch and class axis for calculating Dice score
    axes = tuple(range(1, len(y_pred.shape)-1)) 
    numerator = 2. * K.sum(y_pred * y_true, axes)
    denominator = K.sum(K.square(y_pred) + K.square(y_true), axes)
    
    return 1 - K.mean(numerator / (denominator + epsilon)) # average over classes and batch

## Training

In [0]:
# Most recent iteration for each date, doesn't work anymore because models were moved.

# iterations = dict()
# filenames = dict()
# for filename in os.listdir(f'{root_path}'):
#   match = re.match(r'model-(\d\d\d\d)\-(\d\d)\-(\d\d)\-(\d+)\.p3', filename)
#   if match:
#     year, month, day, iteration = map(int, match.groups())
#     current_iteration = iterations.get((year, month, day), -1)
#     if iteration >= current_iteration:
#       iterations[(year, month, day)] = iteration
#       filenames[(year, month, day)] = filename

# for filename in filenames.values():
#   print(filename)

In [0]:
# train_data = iter_super_batches(train_batches, files_at_once=40, shuffle=True, t=0.8)

In [0]:
# if you wish to create a new model

# i=-1
# model = UNET(n_channels=12)

In [0]:
# if you wish to load an existing model

i=0 # load the save after super batch `i`
model_filename = f'{root_path}/Models/2019-08-13/model-2019-08-13-{i}.p3'

model = load_model(model_filename, {
    'UNET': UNET,
    'normalized_binary_crossentropy': normalized_binary_crossentropy,
    'precision': precision,
    'recall': recall,
    'k_mean': k_mean,
    'k_min': k_min,
    'k_max': k_max,
    'soft_dice_loss': soft_dice_loss
})

In [0]:
model.compile(
    optimizer = Adam(lr = 0.001),
    loss = soft_dice_loss,
    metrics = ['accuracy', precision, recall, k_min, k_max, k_mean]
)

# if you like to train

# for i, (X, y) in enumerate(train_data, start=i+1):
#   curve = model.fit(
#       X, y,
#       batch_size=16,
#       epochs = 50,
#       verbose = 1,
#       shuffle = True,
#       validation_data = (X_valid, y_valid)
#   )
#   model.save(f'{root_path}/model-2019-08-13-{i}.p3')
#   del X, y

## Results

In [46]:
y_pred = model.predict(X_valid, batch_size=8, verbose=1)

284/284 [==============================] - 13s 46ms/step


### Prediction examples

Predict on some of the samples.

In [0]:
axes = pyplot.subplots(8, 2, figsize=(25, 100))[1].ravel()


for x, y, y_pred_, axis in zip(X_valid, y_valid, y_pred, axes):
  rgb = (
    0.5 * numpy.ones(x.shape[:2] + (3,)) + 
    0.5 * (x[..., [3,2,1]] + 2) / 4
  )
  
  nothing = numpy.zeros(y.shape, dtype=numpy.uint8)
  mask = numpy.concatenate([
      255 * y,
      nothing,
      255 * (y_pred_ > .5),
      100 * y + 100 * (y_pred_ > .5),
  ], axis=2).astype(numpy.uint8)
  
  axis.imshow(rgb)
  axis.imshow(mask)
  
pyplot.tight_layout()

pyplot.savefig(f'{results_path}/example_predictions.png')

### False predictions

For sharability, I want to plot all of the false predicitons in one subplot. Hence I create a list of `plotters` in one cell, which contains functions that given an `axis` plot the false prediction. This way I can create the subplot afterwards, because I'll know how many subplot axes are needed.

The image is too big and often won't show :(

#### Helper functions

In [0]:
def connected_component_coordinates(mask):
  """Returns a list with for each connected component in `mask` a sublist of
  the pixel coordinates of that connected component."""
  labels, n = label(mask)

  r = [list() for _ in range(n)]
  for y, row in enumerate(labels):
    for x, elem in enumerate(row):
      if elem > 0:
        r[int(elem - 1)].append((x, y))
  return r


def bbox(coords):
  return (
      min(x for x, y in coords),
      max(x for x, y in coords),
      min(y for x, y in coords),
      max(y for x, y in coords),
  )


def find_false_prediction_centers(y_true, y_pred):
  """ Returns (xmin, xmax, ymin, ymax) bounding boxes for predicted
  (false positives) and actual (false negatives) connected components that do
  not overlap respectively with an actual or a predicted connected component.
  
  @param `y_true` m x n boolean numpy array of ground truth
  @param `y_true` m x n boolean numpy array of ground truth
  @return two lists of bounding boxes for the false negatives and false
          positives respectively."""    
  true_positives = set(sum(
      connected_component_coordinates(
          numpy.minimum(y_true, y_pred)
      ), []
  ))
  
  false_positives = [
    coords
    for coords in connected_component_coordinates(y_pred)
    if not any(coord in true_positives for coord in coords)
  ]
  
  false_negatives = [
    coords
    for coords in connected_component_coordinates(y_true)
    if not any(coord in true_positives for coord in coords)
  ]
  
  return list(map(bbox, false_negatives)), list(map(bbox, false_positives))

#### Calculations

In [0]:
plotters = []
padding = 50

for img, t, p in zip(ProgressBar(X_valid), y_valid, y_pred > 0.5):
  rgb = (
    0.6 * numpy.ones(img.shape[:2] + (3,)) + 
    0.4 * (img[..., [3,2,1]] + 2) / 4
  )
  
  nothing = numpy.zeros(p.shape, dtype=numpy.uint8)
  mask = numpy.concatenate([
      255 * t,
      nothing,
      255 * p,
      70 * numpy.maximum(t, p)
  ], axis=2).astype(numpy.uint8)
  
  def plot(rgb, mask):
    def p(axis):
      axis.imshow(rgb)
      axis.imshow(mask)
    return p
  
  plotters.append(plot(rgb, mask))
  
  false_negatives, false_positives = find_false_prediction_centers(t, p)
  
  error_types = [[1., 1., 0.]] * len(false_negatives) + [[1., 0., 0.]] *  len(false_positives)
  for (x0, x1, y0, y1), color in zip(false_negatives + false_positives, error_types):
    if (
        y0 < padding or y1 >= t.shape[0] - padding or
        x0 < padding or x1 >= t.shape[1] - padding
    ):
      continue
      color[2] = 0.333
    
    w = max(x1-x0, y1-y0)
    wx = max(0, w - x1 + x0) // 2
    wy = max(0, w - y1 + y0) // 2
    y0 = max(0, y0 - padding - wy)
    x0 = max(0, x0 - padding - wx)
    region = (slice(y0, y0 + 2*padding + w), slice(x0, x0 + 2*padding + w))
    
    def plot(rgb, mask, color, w):
      def p(axis):
        axis.imshow(rgb)
        axis.imshow(mask)
        
        for child in axis.get_children():
          if isinstance(child, matplotlib.spines.Spine):
            child.set_color(color)
            child.set_linewidth(4)
          
        axis.set_xlim(0, w); axis.set_ylim(0, w)
      return p
    plotters.append(plot(rgb[region], mask[region], color, w + 2*padding))

#### Result

In [0]:
width = 24
ncols = 8
nrows = int(numpy.ceil(len(plotters) / ncols))
nrows_max = min(8, nrows)

axes = []
figures = []
for _ in range(0, nrows, nrows_max):
  fig, axes_ = pyplot.subplots(
      nrows=nrows_max, ncols=ncols,
      figsize=(width, width * nrows_max / ncols)
  )
  axes.extend(axes_.ravel())
  figures.append(fig)

for axis in axes:
  axis.set_xticks([])
  axis.set_yticks([])

for axis, plotter in zip(axes, ProgressBar(plotters)):
  plotter(axis)

for i, fig in enumerate(figures):
  fig.tight_layout()
  fig.savefig(f'{results_path}/false_predictions_batch-{i}.png')

Output hidden; open in https://colab.research.google.com to view.